# This lab is for use of OpenConfig YANG Models with JunOS

In [1]:
# import ncclient to work with NETCONF to push the configuration
import lxml.etree as ET
from argparse import ArgumentParser
from ncclient import manager
from ncclient.operations import RPCError

In [23]:
# XML RPC to delete Interface using OpenConfig YANG Models
'''
payload = """
<config xmlns:xc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="http://openconfig.net/yang/interfaces">
    <interface xc:operation="delete">
      <name>ge-0/0/2</name>
    </interface>
  </interfaces>
</config>
"""
'''

'\npayload = """\n<config xmlns:xc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">\n  <interfaces xmlns="http://openconfig.net/yang/interfaces">\n    <interface xc:operation="delete">\n      <name>ge-0/0/2</name>\n    </interface>\n  </interfaces>\n</config>\n"""\n'

In [25]:
# We can't use the same model similar to IOS-XE with oc-ip reference
payload = """
<config xmlns:xc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
    <interfaces xmlns="http://openconfig.net/yang/interfaces">
      <interface>
        <name>ge-0/0/1</name>
        <config>
          <type>ianaift:ethernetCsmacd</type>
          <name>ge-0/0/1</name>
          <description>Link to XR5 -- OpenConfig Model</description>
          <enabled>true</enabled>
        </config>
        <subinterfaces>
          <subinterface>
            <index>0</index>
            <config>
              <index>0</index>
              <enabled>true</enabled>
            </config>
            <ipv4 xmlns="http://openconfig.net/yang/interfaces/ip">
              <addresses>
                <address>
                  <ip>172.10.35.1</ip>
                  <config>
                    <ip>172.10.35.1</ip>
                    <prefix-length>30</prefix-length>
                  </config>
                </address>
              </addresses>
            </ipv4>
          </subinterface>
        </subinterfaces>
      </interface>
    </interfaces>
</config>
"""

In [26]:
   with manager.connect(host='192.10.0.103',
                         port=830,
                         username='lab',
                         password='lab123',
                         timeout=90,
                         hostkey_verify=False,
                         device_params={'name': 'default'}) as m:

        # execute netconf operation
        try:
            response = m.edit_config(target='candidate', config=payload).xml
            m.commit()
            data = ET.fromstring(response)
        except RPCError as e:
            data = e._raw

        # beautify output
        print(ET.tostring(data, pretty_print=True))

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/17.1R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:efc635b2-9772-4778-8e33-a938c1b7e9be">
<ok/>
</rpc-reply>



In [33]:
get_interface = """
<filter xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
      <name>ge-0/0/2</name>
</filter>
"""

In [34]:
   with manager.connect(host='192.10.0.103',
                         port=830,
                         username='lab',
                         password='lab123',
                         timeout=90,
                         hostkey_verify=False,
                         device_params={'name': 'default'}) as m:

        # execute netconf operation
        try:
            response = m.get(get_interface).xml
            data = ET.fromstring(response)
        except RPCError as e:
            data = e._raw

        # beautify output
        print(ET.tostring(data, pretty_print=True))

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/17.1R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:0cd0f1ba-8586-42b1-9696-7b7dcc230311">
<rpc-error>
<error-type>protocol</error-type>
<error-tag>operation-failed</error-tag>
<error-severity>error</error-severity>
<error-message>syntax error, expecting &lt;/filter&gt;</error-message>
<error-info>
<bad-element>name</bad-element>
</error-info>
</rpc-error>
<rpc-error>
<error-type>protocol</error-type>
<error-tag>operation-failed</error-tag>
<error-severity>error</error-severity>
<error-message>syntax error, expecting &lt;/filter&gt;</error-message>
<error-info>
<bad-element>name</bad-element>
</error-info>
</rpc-error>
</rpc-reply>

